In [14]:
from Graph import Graph

# initialize graph
graph = Graph()

In [23]:
# retrieve active_player_histories-- this will be used to create connections
import pandas as pd
from datetime import datetime

# initialize players dict for O(1) lookup of player information
players_csv = pd.read_csv("active_players_nba_2023.csv")
players = {}

for row in players_csv.iterrows():
    content = row[1]
    players[content["id"]] = content["name"]


player_trades = pd.read_csv("active_player_histories_nba.csv")
player_trades.fillna(datetime.today().strftime('%Y-%m-%d'), inplace=True)
player_trades.sort_values("end", inplace=True, ascending=False)
player_trades.head(5)
# insertion will be done by generating current teams and replaying trades from present -> past
# the sorting allows this


,Unnamed: 0,player_id,player_name,team,start,end
0,0,817057,DeMar DeRozan,Chicago Bulls,2021-08-02,2023-04-16
697,0,817022,Clint Capela,Atlanta Hawks,2020-02-04,2023-04-16
700,1181233,1181233,AJ Griffin,Atlanta Hawks,2023-04-16,2023-04-16
701,0,1133787,Jalen Johnson,Atlanta Hawks,2021-07-28,2023-04-16
702,0,987933,Jarrett Culver,Memphis Grizzlies,2021-08-16,2023-04-16


In [18]:
current_date = None
teams = {}
current_team = {}

def add_to_team(player, team):
    # init team if not initialized
    teams[team] = teams.get(team, set())

    # remove player from their current team
    curr_team = current_team.get(player)
    if curr_team != None:
       teams[curr_team].remove(player)

    # set player current team
    current_team[player] = team

    # add player to team
    teams[team].add(player)

def connect_players(players):
    players = list(players) # convert to an array, so we can index[0]

    for i in range(len(players)):
        curr_player = players[i]
        for j in range(i + 1, len(players)):
            connect_player = players[j]
            graph.addNeighbor(curr_player, connect_player)

# this connects all teams at their current state
def flush_teams():
    global current_date
    for _, players in teams.items():
        connect_players(players)
    current_date = None

# iterate through all player trades
for row in player_trades.iterrows():
    content = row[1]
    date = content["end"]

    if current_date != date:
         flush_teams()

    if current_date == None:
        current_date = date
    
    add_to_team(content["player_id"], content["team"])



-- Graph contents:
  817057: {885248, 885250, 885251, 940804, 1181314, 817412, 1181319, 940809, 942986, 817423, 861455, 1092881, 1132153, 1092880, 817172, 987552, 987559, 817063, 817191, 817068, 817069, 817196, 1179822, 855985, 817458, 817332, 817206, 817463, 1092921, 963001, 1179581, 817218, 817352, 954825, 1092937, 817106, 1092947, 940758, 989015, 1133911, 885207, 1133914, 817240, 817239, 1133917, 816989, 1133784, 1092954, 885217, 816990, 1133795, 940778, 1133165, 846963, 846964, 846965, 817270, 957172, 855801, 885242, 846971, 846972, 846973, 817406}
  817218: {940800, 817025, 817410, 885251, 940804, 972550, 1181319, 817160, 940809, 817422, 817423, 861455, 1092881, 1132153, 988302, 905877, 817057, 817442, 817317, 817063, 905898, 1000618, 817068, 817069, 817196, 1181226, 817328, 855985, 990002, 817329, 951089, 817204, 817463, 1092921, 963001, 1092929, 1092930, 954825, 817354, 846933, 817237, 989015, 1133911, 885207, 1133914, 817238, 1133917, 816989, 987872, 885217, 940774, 846963, 855

In [32]:
import networkx as nx 
networkG = nx.Graph(graph.connections)
pos = nx.spring_layout(networkG, k=.2)

output = {
    "edges": [],
    "nodes": []
}

max_edges = 0

# fill edges
for source, targets in graph.connections.items():
    max_edges = max(max_edges, len(targets))
    for target in targets:
        output["edges"].append({
            "source": source,
            "target": target,
        })


largest_node_size = 7.5


# fill nodess
for key, value in pos.items():
    output["nodes"].append({
        "key": key,
        "attributes": {
            "x": value[0],
            "y": value[1],
            "size": len(graph.connections[key])/max_edges * largest_node_size,
            "label": players[key]
        }
    })

import json

with open("../data.json", "w") as outfile:
    json.dump(output, outfile)
